In [1]:
import os
os.chdir("../VITAL")

from openai import AzureOpenAI

api_key = "bffeba6e73e24113bf6cd0457b0360f3"
azure_endpoint = "https://rtp2-shared.openai.azure.com/openai/deployments/o4-mini/chat/completions?api-version=2025-01-01-preview"
model_name = "o4-mini"

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    api_version="2025-01-01-preview",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
)

A = 10
import json, random
def get_aug_dict(context, str_cols, df_train):
    text_dict = {}
    for str_col in str_cols:
        text_dict[str_col] = {}
        for unique_str in list(df_train[str_col].astype(str).unique()):
            # augment this unique_str for A times, using openai
            prompt = f"With simple changes, generate {A} different sentences that has the same meaning as this sentence related to {context}: '{unique_str}'. Split by '|' and return the list of sentences."
            completion = client.chat.completions.create(
                model=model_name,
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    },
                ],
            )
            response = json.loads(completion.model_dump_json(indent=2))['choices'][0]['message']['content']
            aug_strs = response.split('|')
            aug_strs = [s.strip() + ('' if s.strip().endswith('.') else '.') for s in aug_strs if s.strip()] # # Ensure each string ends with a period
            text_dict[str_col][unique_str] = random.sample(aug_strs, k=A) # shuffle
    return text_dict


# Air quality

In [2]:
overwrite = False
dataset_name = 'air'
attr_suffix = ''
suffix = '_self' 
with open('run/settings.py', 'r') as file:
    exec(file.read())
with open('run/data.py', 'r') as file:
    exec(file.read())

Random seed set to 333
using device:  cpu
air_self
air_self


final distribution of text prediction
text
This is air quality in Beijing. The season is winter. It is measured in 2017.    282
This is air quality in Beijing. The season is fall. It is measured in 2017.      246
This is air quality in Beijing. The season is spring. It is measured in 2017.    202
This is air quality in Beijing. The season is summer. It is measured in 2017.    195
This is air quality in London. The season is spring. It is measured in 2017.     179
This is air quality in Beijing. The season is winter. It is measured in 2018.    173
This is air quality in London. The season is winter. It is measured in 2017.     156
This is air quality in London. The season is fall. It is measured in 2017.       143
This is air quality in London. The season is summer. It is measured in 2017.     126
This is air quality in London. The season is winter. It is measured in 2018.      88
This is air quality in Beijing. The season is

In [3]:
text_dict = get_aug_dict(
    context  = 'air quality', 
    str_cols = ['city_str', 'year_str', 'season_str'],
    df_train = df_train
)
with open("../../data/air_quality/aug_text.json", "w") as f:
    json.dump(text_dict, f, indent=2, ensure_ascii=False)

# Synthetic

In [4]:
overwrite = False
dataset_name = 'syn'
attr_suffix = ''
suffix = '_self' 
with open('run/settings.py', 'r') as file:
    exec(file.read())
with open('run/data.py', 'r') as file:
    exec(file.read())

using device:  cpu
syn_self
syn_self
text
No trend. No seasonal pattern. No sharp shifts. The time series exhibits low variability.                                                                                                10000
No trend. No seasonal pattern. No sharp shifts. The time series exhibits high variability.                                                                                               10000
The time series shows downward linear trend. The time series exhibits a seasonal pattern. The mean of the time series shifts upwards. The time series exhibits low variability.          10000
The time series shows downward linear trend. The time series exhibits a seasonal pattern. The mean of the time series shifts upwards. The time series exhibits high variability.         10000
The time series shows downward linear trend. The time series exhibits a seasonal pattern. The mean of the time series shifts downwards. The time series exhibits low variability.        10000
The

In [5]:
text_dict = get_aug_dict(
    context  = 'time series', 
    str_cols = ['segment1', 'segment2', 'segment3', 'segment4'],
    df_train = df_train
)
with open("../../data/synthetic/aug_text.json", "w") as f:
    json.dump(text_dict, f, indent=2, ensure_ascii=False)

# NICU

In [6]:
overwrite = False
dataset_name = 'nicu'
attr_suffix = ''
suffix = '_self' 
with open('run/settings.py', 'r') as file:
    exec(file.read())
with open('run/data.py', 'r') as file:
    exec(file.read())

using device:  cpu
nicu_self
nicu_self

Sample of patients with positive labels:
VitalID
1018    8
5170    8
1835    8
2361    8
2791    8
dtype: int64


Processing descriptions: 100%|██████████| 131/131 [00:28<00:00,  4.53it/s]


After downsampling:
description_histogram
High variability.        10000
Moderate variability.    10000
Low variability.         10000
Name: count, dtype: int64
train, test, left:  21000 6000 3000


final distribution of text prediction
description_histogram
High variability.        7000
Low variability.         7000
Moderate variability.    7000
Name: count, dtype: int64
description_histogram
High variability.        2000
Moderate variability.    2000
Low variability.         2000
Name: count, dtype: int64
description_histogram
Low variability.         1000
High variability.        1000
Moderate variability.    1000
Name: count, dtype: int64
text
Low variability. No events.                                                     6788
Moderate variability. No events.                                                6650
High variability. No events.                                                    6342
High variability. Bradycardia 100 event (heart rate below 100) happened.         342
High

In [7]:
text_dict = get_aug_dict(
    context  = 'NICU heart rate', 
    str_cols = ['description_succ_inc', 'description_histogram', 'description_ts_event', 'description_ts_event_binary'],
    df_train = df_train
)
with open("../../data/nicu/aug_text.json", "w") as f:
    json.dump(text_dict, f, indent=2, ensure_ascii=False)

In [8]:
text_dict

{'description_succ_inc': {'NA': ['Heart rate data is not available.',
   'The heart rate is not available.',
   'No heart rate measurement is available.',
   'Heart rate measurement is not recorded.',
   'Heart rate data unavailable.',
   'Heart rate measurement is unavailable.',
   'No heart rate data recorded.',
   'Heart rate is unavailable.',
   'Heart rate not applicable.',
   'Heart rate information is not available.'],
  'Low amount of consecutive increases.': ['A low number of consecutive heart rate rises.',
   'Reduced series of consecutive heart rate elevations.',
   'An insufficient number of successive heart rate increases.',
   'Few successive increases in heart rate.',
   'Limited consecutive heart rate upticks.',
   'Sparse back-to-back increases in heart rate.',
   'A small count of back-to-back heart rate increases.',
   'Scarce successive heart rate elevations.',
   'Minimal sequential rises in heart rate.',
   'A limited frequency of consecutive heart rate increases.